In [1]:
#!pip install pyproj
#!pip install request

# MANUAL SKETCHBOOK

La finalidad de este software es completar la mayor cantidad de información del estandar Darwin Core a partir de un par de coordenadas, *importante hacer notar que este software hasta el momento no realiza ningun tipo de georreferenciación, solo rellena datos*, esto se realiza a partir de un set de coordenadas, esta pueden ser del campo verbatimCoordinates o decimalLatitude y decimalLongitude, pertenecientes al estandar. Con estos datos será completar los siguientes campos, si es que se encuentra la información necesaria. 

- higherGeography 
- continent 
- country 
- countryCode 
- stateProvince 
- county 
- municipality 
- decimalLongitude, decimalLatitude
- geodeticDatum 
- coordinatePrecision 
- verbatimLatitude, verbatimLongitude
- verbatimCoordinateSystem 

Hasta el momento no se ha podido probar la localización de localidades marinas, pero es un trabajo en progreso. 

Este script se divide en dos clases

- Preparación de la base de datos (db_prep)
- Transformación de coordenadas (coord_transform) 

## Preparación de la base de datos

Esta clase recibe el nombre de "db_prep" y necesita como entrada una base de datos con algunos componentes mínimos del estandar Darwin Core (DwC) para su realización.



## Transformación de coordenadas 

- El campo verbatimCoordintes debe tener las coordenadas en el siguiente formato (dejar en georeferenceRemarks las coordenadas como vienen en su etiqueta original)
        -En el caso de coordenadas en grados minutos y segundos gg°mm'ss" S, gg°mm'ss" W o gg°mm'ss" N, gg°mm'ss" E la importancia radica que debes estar separadas por una coma, misma situación para grados y minutos decimales
        -En el caso de UTM deben ser escritas de la siguiente forma 308617 6272156 19S, la importancia esta que se encuentren separadas por espacio. 
    

In [2]:
from GeoParser import *
import pandas as pd

In [3]:
data=pd.read_csv("GeoRefTest.csv")
data

,higherGeography,dynamicProperties,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,...,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid
0,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TxmVXS0HuObUcicDzF9JBaWNHK4C/tiPdf9jxpTLMqU=
1,NaN,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,WRYd0YGi5ik2bvvJRhC1Ynf4OyfzirN66qbhuD3Fgfc=
2,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TtMBF2hQMfzrgMVqyIbTHRP0UF9v4Qn8uNonMtuye/s=
3,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,gSE0qlewC1j4UoKOU2+A5amKtzc1j/I5h2bhjv2d+Q8=
4,NaN,"{""textFieldMeasurementError"": 0.005, ""ChoiceDi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,Tw11N7qFI/XzbJam2Z1HHIoLjzOfoRsDpZEPw5+W4tU=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,rbPp7LlP8R78/VBHQiWfqA51+qeeu62rJ8XODXqBm/c=
106,NaN,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.geonames.org | Google Earth | http...,NaN,Georreferenciado a través de mapas de la Bibli...,HVoHXv8ifqVeUCAYhiDRTuyKmqixjppKV3bwVx7MAbw=
107,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2018-09-07,NaN,NaN,NaN,En base a la localidad,YCgWdFqJbtrW1Yv4keFu6Ld+IpjfOChW5WP7u/oYAoM=
108,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,/0h1tIuArvSEx1ktY3JjnXCcHGrefNvWljHHsTqfcAs=


In [4]:
%%time
#Testing...
DBPrep(data).verbatim_coordinates_autocomplete()
DBPrep(data).verbatim_coordinate_system_autocomplete()
DBPrep(data).verbatim_latitude_longitude_autocomplete()
#DBPrep(data).coordinate_precision_autocomplete() Deprecated function implemented on GeorefCalculator

Wall time: 31 ms


In [5]:
data_pep=data.query("decimalLatitude.notna() and decimalLongitude.notna() ")
for index in data_pep.index:
    verbatimLatitude=data_pep.loc[index,"decimalLatitude"]
    verbatimLongitude=data_pep.loc[index,"decimalLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem="decimal degrees"
    verbatimSRS=data_pep.loc[index,"geodeticDatum"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
    

In [6]:
data_pep=data.query("verbatimLatitude.notna() and verbatimLongitude.notna() ")
for index in data_pep.index:
    verbatimLatitude=data_pep.loc[index,"verbatimLatitude"]
    verbatimLongitude=data_pep.loc[index,"verbatimLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem=data_pep.loc[index,"verbatimCoordinateSystem"]
    verbatimSRS=data_pep.loc[index,"verbatimSRS"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]

In [7]:
%%time
# Testing ...
#coord_transform(data).utm_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).dms_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).geodetic_datum_autocomplete()  Deprecated function implemented on GeorefCalculator
CoordTransform(data).location_autocomplete() 
CoordTransform(data).continent_autocomplete() 
CoordTransform(data).higher_geography_autocomplete() 

Wall time: 1min 54s


In [8]:
driver.quit()

In [9]:
data

,higherGeography,dynamicProperties,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,...,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid
0,South America | Chile | CL | Araucanía | Malle...,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,South America,Chile,CL,Araucanía,Malleco,Lonquimay,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:49:09.104Z,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TxmVXS0HuObUcicDzF9JBaWNHK4C/tiPdf9jxpTLMqU=
1,South America | Chile | CL | Ñuble | Punilla |...,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,South America,Chile,CL,Ñuble,Punilla,Coihueco,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:49:10.201Z,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,WRYd0YGi5ik2bvvJRhC1Ynf4OyfzirN66qbhuD3Fgfc=
2,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:49:11.098Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TtMBF2hQMfzrgMVqyIbTHRP0UF9v4Qn8uNonMtuye/s=
3,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:49:11.889Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,gSE0qlewC1j4UoKOU2+A5amKtzc1j/I5h2bhjv2d+Q8=
4,South America | Chile | CL | Araucanía | Malle...,"{""textFieldMeasurementError"": 0.005, ""ChoiceDi...",NaN,South America,Chile,CL,Araucanía,Malleco,Victoria,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:49:12.864Z,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,Tw11N7qFI/XzbJam2Z1HHIoLjzOfoRsDpZEPw5+W4tU=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,South America | Chile | CL | Región Metropolit...,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,South America,Chile,CL,Región Metropolitana de Santiago,Valparaíso,Hijuelas,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:50:06.840Z,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,rbPp7LlP8R78/VBHQiWfqA51+qeeu62rJ8XODXqBm/c=
106,South America | Chile | CL | Valparaíso | Valp...,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,South America,Chile,CL,Valparaíso,Valparaíso,Valparaiso,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:50:07.701Z,NaN,https://www.geonames.org | Google Earth | http...,NaN,Georreferenciado a través de mapas de la Bibli...,HVoHXv8ifqVeUCAYhiDRTuyKmqixjppKV3bwVx7MAbw=
107,South America | Chile | CL | Valparaíso | ísla...,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,South America,Chile,CL,Valparaíso,ísla de Pascua,Ocean Islands,NaN,...,"78°51'0.000"" W",degrees minutes seconds,NaN,NaN,2021-03-20T20:50:19.963Z,NaN,NaN,NaN,En base a la localidad,YCgWdFqJbtrW1Yv4keFu6Ld+IpjfOChW5WP7u/oYAoM=
108,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-03-20T20:50:08.547Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,/0h1tIuArvSEx1ktY3JjnXCcHGrefNvWljHHsTqfcAs=
